## Auxiliary steps and makefile-style workflows

Auxiliary steps are special steps that are executed to provide [targets](Understanding_Targets.html) that are required by others.

For example, when the following step is executed with an input file `bamfile` (with extension `.bam`), it checks the existence of input file (`bamfile`), and a dependent index file (with extension `.bam.bai`).

```sos
[100 (call variant)]
input:   bamfile
depends: bamfile + '.bai'
run:
    # commands to call variants from 
    # input bam file
```

If the index file exists, either by another step or outside of SoS, sos will go ahead and execute the step. Otherwise  SoS will look in the script for a step that provides such a target, which would be similar to 

```sos
[index_bam : provides='{sample}.bam.bai']
input: '${sample}.bam'
run:
     samtools index ${input}
```

Such a step is defined by the **`provides`** option (or a **`shared`** option that will discuss later) and is called an auxiliary step. In this particular case, if `bamfile="AS123.bam"`, the requested file would be `AS123.bam.bai`. Through the matching mechanism of option `provides`, the `index_bam` step would be executed with variable `sample="AS123"` and `output=["AS123.bam.bai"]`.

An auxiliary step can trigger other auxiliary steps that form a DAG (Directed Acyclic Graph). Acutually, you can write workflows in a make-file style with all auxiliary steps and execute workflows defined by targets. If you are familiar with Makefile, especially [snakemake](https://bitbucket.org/johanneskoester/snakemake), you will find it easy to implement your workflow in this style. The advantage of SoS is that **you can use either or both forward-style and makefile-style steps to define your workflow** and take advantages of both approaches. For example, people frequently need to create fake targets to trigger steps that do not produce any target in a makefile-style workflow system, but do not have to do this in SoS because steps in forward-style will always be executed. 

## Step option `provides`

An auxiliary step can be defined in the format of

```python
[step_name : provides=pattern]
```

where `pattern` can be

* A file pattern such as `"{sample}.bam.idx"`
* Other types of targets such as `executable("ms")`
* A list (sequence) of one or more file patterns and targets.

### File pattern

A file pattern is a filename with optional patterns with variable names enbraced in `{ }`. SoS matches filenames with the patterns and, if successful, assign variables with matched parts of the names. For example,

In [2]:
%set -v1
![ -d tmp_dir ] || mkdir tmp_dir
!rm -f AS123.bam AS123.bam.bai tmp_dir/input.txt

[matched: provides = '{filename}.{ext}']
print("filename: ${filename!r}, ext: ${ext!r}, output: ${repr(output)}")
sh:
    touch ${output}

[10]
sos_run(targets='AS123.bam')
sos_run(targets='AS123.bam.bai')
sos_run(targets='tmp_dir/input.txt')


sos options is set to "-v1"
filename: 'AS123', ext: 'bam', output: ['AS123.bam']
filename: 'AS123.bam', ext: 'bai', output: ['AS123.bam.bai']
filename: 'tmp_dir/input', ext: 'txt', output: ['tmp_dir/input.txt']


## -- Preview output --

input: 
output: tmp_dir/input.txt

> tmp_dir/input.txt (0 B):